In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import numpy as np
import soundfile as sf
import librosa
import IPython.display as ipd
from sklearn.preprocessing import minmax_scale
from glob import glob


In [7]:
voice_file = 'C:/Users/Toby/Speech_Data/LibriSpeech/Mixed/train-clean-100/19/198/19-198-0012.flac'
background_file = 'C:/Users/Toby/Speech_Data/CHiME3/data/audio/16kHz/backgrounds/BGD_150203_010_CAF.CH1.wav' 

In [13]:
def mix_audio(voice_file, background_file):
    
    # Load the files
    voice, voice_sr = sf.read(voice_file)
    background, background_sr = sf.read(background_file)

    if voice_sr != background_sr:
        librosa.resample(background, voice_sr)

    # Select a segment of the background track to use
    start_point = np.random.randint(len(background) - len(voice))
    background_segment = background[start_point:start_point+len(voice)]

    # Normalise both sources and mix
    norm_voice = minmax_scale(voice, (-1,1))
    norm_bgs = minmax_scale(background_segment,(-1,1))
    norm_mix = (0.6*norm_voice) + (0.4*norm_bgs)
    
    return norm_mix, voice_sr, background_segment, voice

In [12]:
# Copy the file structure

inputpath = 'C:/Users/Toby/Speech_Data/BG_test/Voice/'
outputpath = 'C:/Users/Toby/Speech_Data/BG_test/Background/'

for dirpath, dirnames, filenames in os.walk(inputpath):
    structure = os.path.join(outputpath, dirpath[len(inputpath):])
    if not os.path.isdir(structure):
        os.mkdir(structure)
    else:
        print("Folder does already exits!")

In [16]:
background_file_list = glob('C:/Users/Toby/Speech_Data/CHiME3/data/audio/16kHz/backgrounds/*.wav')
directory = 'C:/Users/Toby/Speech_Data/BG_test/Voice/'
file_list = glob(directory+'/**/*.flac', recursive=True)
count = 0

# Loop over files
for file in file_list:
    # Check if file already exists
    split_path = file.split('Voice')
    new_mix_file_path = split_path[0] + 'Mixed' + split_path[1]
    new_bg_file_path = split_path[0] + 'Background' + split_path[1]
    if os.path.isfile(new_mix_file_path):
        print('File already exists!')
        count += 1
        continue
    # Select a background file
    background_file = background_file_list[np.random.randint(len(background_file_list))]
    # Mix them
    mixture, sample_rate, background_noise, _ = mix_audio(file, background_file)
    # Save to file
    sf.write(new_mix_file_path, mixture, sample_rate)
    sf.write(new_bg_file_path, background_noise, sample_rate)
    count += 1
    if count % 1000 == 0:
        print('{c} files processed.'.format(c=count))


In [12]:
mixture, sample_rate, background, voice = mix_audio(voice_file, background_file)

In [13]:
ipd.Audio(background, rate=sample_rate)

In [28]:
ref_sources = np.concatenate((np.expand_dims(mixture, axis=1), np.expand_dims(voice, axis=1)), axis=1)
est_sources = np.concatenate((np.expand_dims(background, axis=1), np.expand_dims(voice, axis=1)), axis=1)

In [26]:
import mir_eval

In [29]:
mir_eval.separation.bss_eval_sources(ref_sources.T, est_sources.T, compute_permutation=False)

C:\Program Files\Anaconda3\lib\site-packages\scipy\fftpack\basic.py:159: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x


(array([ -2.6780796 , 296.28947121]),
 array([ -2.67170637, 264.13017069]),
 array([ 30.20778981, 264.1331665 ]),
 array([0, 1]))